## Benchmarking guide

In this practical tutorial, we showcase how the benchmark database is structured. Furthermore, we exemplify how interested users can evaluate their own custom molecular feature attribution techniques. Before we begin, please make sure that you have followed the installation instructions in the `README.md` file of the repository, particularly those that involve environment setup and database download.

In [1]:
# If you had not explicitly added the graph-attribution repository to your PYTHONPATH, you should do so now.
# Same goes for the root folder of this repository.

import os
import sys

os.chdir("../../")

from xaibench.utils import ROOT_PATH

sys.path.append(os.path.join(os.path.dirname(ROOT_PATH), "graph-attribution"))

All related benchmark data (`data.tar.gz`) should be downloaded and extrated in the root folder of the repository. Data can be then accessed by importing the `BENCHMARK_PATH` variable from the `utils` module. 

In [2]:
from xaibench.utils import BENCHMARK_PATH

all_series = os.listdir(BENCHMARK_PATH)


For simplicity, we will focus the data related to a single congeneric series considered in the benchmark.

In [3]:
idx = 42
series_id = all_series[idx]
print(series_id)


1NAV-IH5


We explore the contents of this subfolder to find all its related data. Most of it corresponds to the assigned colors of the different feature attribution methods considered in this study. For method development, we mostly care about 3 files:

* `pairs.csv`, which contains all benchmark pairs for this specific series.
* `colors.pt`, where the assigned ground truth color determined for each pair in `pairs.csv` is stored.
* `training.csv` contains all training data extracted for this series, possibly containing some of the compounds in `pairs.csv`. A version excluding the latter can be found in `training_wo_pairs.csv`.

In [4]:
series_path = os.path.join(BENCHMARK_PATH, series_id)
os.listdir(series_path)

['training.csv',
 'colors_dnn.pt',
 'colors_dnn_wo_pairs.pt',
 'colors_mpnn.pt',
 'colors_gcn.pt',
 'colors_rf.pt',
 'colors_mpnn_wo_pairs.pt',
 'colors_gat.pt',
 'colors_graphnet.pt',
 'assay_ids.npy',
 'colors_gat_wo_pairs.pt',
 'bench.csv',
 'similarity_wo_pairs.npy',
 'colors_graphnet_wo_pairs.pt',
 '1NAV-IH5.tsv',
 'colors_rf_wo_pairs.pt',
 'colors_gcn_wo_pairs.pt',
 'colors.pt',
 'similarity.npy',
 'pairs.csv',
 'training_wo_pairs.csv']

We here show the contents of the `pairs.csv` file, with each pair of compounds as well as their log10 activity difference per row.  

In [5]:
import pandas as pd

pairs_df = pd.read_csv(os.path.join(series_path, "pairs.csv"))
pairs_df


,smiles_i,smiles_j,diff
0,O=C(O)Cc1cc(Cl)c(Oc2ccc(O)cc2)c(Cl)c1,O=C(O)Cc1cc(Cl)c(Oc2ccc(O)c(Br)c2)c(Cl)c1,-1.519479
1,O=C(O)Cc1cc(Cl)c(Oc2ccc(O)cc2)c(Cl)c1,CC(C)c1cc(Oc2c(Cl)cc(C(=O)O)cc2Cl)ccc1O,-1.442962
2,O=C(O)Cc1cc(Cl)c(Oc2ccc(O)cc2)c(Cl)c1,CC(C)c1cc(Oc2c(Br)cc(CC(=O)O)cc2Br)ccc1O,-3.366574
3,O=C(O)Cc1cc(Cl)c(Oc2ccc(O)cc2)c(Cl)c1,CC(C)c1cc(Oc2c(Cl)cc(CC(=O)O)cc2Cl)ccc1O,-2.158965
4,O=C(O)Cc1cc(Cl)c(Oc2ccc(O)cc2)c(Cl)c1,O=C(O)Cc1cc(Cl)c(Oc2ccc(O)c(-c3cncnc3)c2)c(Cl)c1,-1.126911
...,...,...,...
122,O=C(O)Cc1cc(Cl)c(Oc2ccc(O)c(-c3ccc(C(F)(F)F)cc...,O=C(O)Cc1cc(Cl)c(Oc2ccc(O)c(-c3cccc(OC(F)F)c3)...,-1.231800
123,O=C(O)Cc1cc(Cl)c(Oc2ccc(O)c(-c3ccc(C(F)(F)F)cc...,CCc1cccc(-c2cc(Oc3c(Cl)cc(CC(=O)O)cc3Cl)ccc2O)c1,-1.597864
124,O=C(O)Cc1cc(Cl)c(Oc2ccc(O)c(-c3cccc(OC(F)F)c3)...,O=C(O)Cc1cc(Cl)c(Oc2ccc(O)c(-c3cccc(-c4ccccc4)...,1.179995
125,COc1cccc(-c2cc(Oc3c(Cl)cc(CC(=O)O)cc3Cl)ccc2O)c1,CCc1cccc(-c2cc(Oc3c(Cl)cc(CC(=O)O)cc3Cl)ccc2O)c1,-1.358813


In the serialised `colors.pt` file we can find ground-truth information for each of the ligands in the `pairs.csv` file. Note that a total of 10 different MCS percentage thresholds were used in the study (from `0.5` to `0.95` in increments of `0.05`). This information can be accessed through the second index of the object. 

Once a sample_idx and a threshold has been selected, each item in `colors.pt` consists of a tuple of dictionaries, for ligands i and j in `pairs.csv`, respectively. These dictionaries are keyed with their atomic indexes as determined with rdkit's `MolFromSmiles` reader, and their values marking whether each specific atom is part of the MCS (`0.0`), or whether it is colored positively (`1.0`) because it contributes to a increase in activity in the pair, or viceversa (`-1.0`).

In [6]:
import dill

with open(os.path.join(series_path, "colors.pt"), "rb") as handle:
    colors_true = dill.load(handle)
len(colors_true)


127

In [7]:
idx_mcs_threshold = 0
len(colors_true[0])


10

In [8]:
colors_true[idx][idx_mcs_threshold]


({0: 0.0,
  1: 0.0,
  2: 0.0,
  3: 0.0,
  4: 0.0,
  5: 0.0,
  6: 0.0,
  7: 0.0,
  8: 0.0,
  9: 1.0,
  10: 0.0,
  11: 0.0,
  12: 0.0,
  13: 0.0,
  14: 0.0,
  15: 0.0,
  16: 0.0,
  17: 0.0,
  18: 1.0,
  19: 0.0,
  20: 0.0,
  21: 0.0,
  22: 0.0},
 {0: 0.0,
  1: 0.0,
  2: 0.0,
  3: 0.0,
  4: 0.0,
  5: 0.0,
  6: 0.0,
  7: -1.0,
  8: 0.0,
  9: 0.0,
  10: 0.0,
  11: 0.0,
  12: 0.0,
  13: 0.0,
  14: 0.0,
  15: 0.0,
  16: 0.0,
  17: 0.0,
  18: -1.0,
  19: -1.0,
  20: -1.0,
  21: -1.0,
  22: -1.0,
  23: -1.0,
  24: -1.0,
  25: 0.0,
  26: 0.0,
  27: 0.0,
  28: -1.0,
  29: 0.0})

Assuming that the user wants to develop their custom molecular attribution function, exemplified below by the `your_custom_molfa` function, it needs to output a numpy array with an atom importance in the same order as the corresponding rdkit atom iterator. We proceed and compute the toy molecular feature attribution function in the `pairs.csv` file.

In [9]:
import numpy as np
from rdkit.Chem import MolFromSmiles

np.random.seed(42)

def your_custom_molfa(smiles):
    mol = MolFromSmiles(smiles)
    return np.random.uniform(
        low=-1, high=1, size=mol.GetNumAtoms()
    )  # random assignment


your_custom_molfa(pairs_df["smiles_i"][0])


array([-0.25091976,  0.90142861,  0.46398788,  0.19731697, -0.68796272,
       -0.68801096, -0.88383278,  0.73235229,  0.20223002,  0.41614516,
       -0.95883101,  0.9398197 ,  0.66488528, -0.57532178, -0.63635007,
       -0.63319098, -0.39151551,  0.04951286, -0.13610996, -0.41754172])

In [10]:
colors_pred = [
    (your_custom_molfa(sm_i), your_custom_molfa(sm_j))
    for sm_i, sm_j in zip(pairs_df["smiles_i"], pairs_df["smiles_j"])
]



In [11]:
colors_pred[0]


(array([ 0.22370579, -0.72101228, -0.4157107 , -0.26727631, -0.08786003,
         0.57035192, -0.60065244,  0.02846888,  0.18482914, -0.90709917,
         0.2150897 , -0.65895175, -0.86989681,  0.89777107,  0.93126407,
         0.6167947 , -0.39077246, -0.80465577,  0.36846605, -0.11969501]),
 array([-0.75592353, -0.00964618, -0.93122296,  0.8186408 , -0.48244004,
         0.32504457, -0.37657785,  0.04013604,  0.09342056, -0.63029109,
         0.93916926,  0.55026565,  0.87899788,  0.7896547 ,  0.19579996,
         0.84374847, -0.823015  , -0.60803428, -0.90954542, -0.34933934,
        -0.22264542]))

The analysis functions that were used in the manuscript are available under the `score` module.

* `color_agreement` computes a separate atom-level color agreement metric on for each molecule of the pair (Figure 4).
* `aggregated_color_direction` checks whether the average non-MCS atomic contributions of each molecule in the pair agrees with the direction of the activity difference sign (Figure 5).

In [12]:
from xaibench.score import color_agreement, aggregated_color_direction
from sklearn.metrics import accuracy_score, f1_score
f_score = lambda x, y: f1_score(x, y, zero_division=1)


acc_scores_i = [color_agreement(colors_true[idx][idx_mcs_threshold][0], colors_pred[idx][0], accuracy_score) for idx in range(len(pairs_df))]
acc_scores_j = [color_agreement(colors_true[idx][idx_mcs_threshold][1], colors_pred[idx][1], accuracy_score) for idx in range(len(pairs_df))]
acc_scores = acc_scores_i + acc_scores_j

f1_scores_i = [color_agreement(colors_true[idx][idx_mcs_threshold][0], colors_pred[idx][0], f_score) for idx in range(len(pairs_df))]
f1_scores_j = [color_agreement(colors_true[idx][idx_mcs_threshold][1], colors_pred[idx][1], f_score) for idx in range(len(pairs_df))]
f1_scores = f1_scores_i + f1_scores_j

rdkit detected? True


Some caution needs to be taken when using the `color_agreement` function, as it will output `-1.0` in cases where all the assigned ground truth atomic values have a value of `0.0`, which corresponds to the case where one the molecules in the pair is a substructure of the other. Therefore these values need to be filtered out before any final metric can be computed.

In [13]:
acc_scores = np.array(acc_scores)
acc_scores = acc_scores[acc_scores >= 0.0]

f1_scores = np.array(f1_scores)
f1_scores = f1_scores[f1_scores >= 0.0]

print(np.mean(acc_scores), np.mean(f1_scores))

0.4448728354978355 0.3477144259510331


In [14]:
agg_direction_scores = [
    aggregated_color_direction(
        colors_pred[idx][0],
        colors_pred[idx][1],
        colors_true[idx][idx_mcs_threshold][0],
        colors_true[idx][idx_mcs_threshold][1],
        pairs_df["diff"][idx],
    )
    for idx in range(len(pairs_df))
]

print(np.mean(agg_direction_scores))

0.4251968503937008
